In [1]:
import pandas as pd
import mysql.connector
from datetime import datetime

In [2]:
file_path = r"D:\GUVI\Project\traffic_stops - traffic_stops_with_vehicle_number.csv"
df = pd.read_csv(file_path)
df

C:\Users\Rajkumar\AppData\Local\Temp\ipykernel_8480\1321501746.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,NaN,Warning,True,0-15 Min,True,KA24UZ8488


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65538 entries, 0 to 65537
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   stop_date           65538 non-null  object
 1   stop_time           65538 non-null  object
 2   country_name        65538 non-null  object
 3   driver_gender       65538 non-null  object
 4   driver_age_raw      65538 non-null  int64 
 5   driver_age          65538 non-null  int64 
 6   driver_race         65538 non-null  object
 7   violation_raw       65538 non-null  object
 8   violation           65538 non-null  object
 9   search_conducted    65538 non-null  bool  
 10  search_type         43818 non-null  object
 11  stop_outcome        65538 non-null  object
 12  is_arrested         65538 non-null  bool  
 13  stop_duration       65538 non-null  object
 14  drugs_related_stop  65538 non-null  bool  
 15  vehicle_number      65538 non-null  object
dtypes: bool(3), int64(2), 

In [4]:
df.isnull().sum()

stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age_raw            0
driver_age                0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64

In [5]:
df['stop_date'] = pd.to_datetime(df['stop_date'])
df['stop_time'] = pd.to_datetime(df['stop_time'] , format='%H:%M:%S').dt.time
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65538 entries, 0 to 65537
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   stop_date           65538 non-null  datetime64[ns]
 1   stop_time           65538 non-null  object        
 2   country_name        65538 non-null  object        
 3   driver_gender       65538 non-null  object        
 4   driver_age_raw      65538 non-null  int64         
 5   driver_age          65538 non-null  int64         
 6   driver_race         65538 non-null  object        
 7   violation_raw       65538 non-null  object        
 8   violation           65538 non-null  object        
 9   search_conducted    65538 non-null  bool          
 10  search_type         43818 non-null  object        
 11  stop_outcome        65538 non-null  object        
 12  is_arrested         65538 non-null  bool          
 13  stop_duration       65538 non-null  object    

In [6]:
df = df.drop('driver_age_raw' , axis=1)
df = df.drop('violation_raw' , axis=1)
df

,stop_date,stop_time,country_name,driver_gender,driver_age,driver_race,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,00:00:00,Canada,M,19,Asian,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,00:01:00,India,M,58,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,00:02:00,USA,M,76,Black,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,00:03:00,Canada,M,76,Black,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,00:04:00,Canada,M,75,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,48,Black,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568
65534,2020-02-15,12:14:00,Canada,F,35,Hispanic,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098
65535,2020-02-15,12:15:00,USA,M,41,Asian,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328
65536,2020-02-15,12:16:00,Canada,F,63,Black,Other,False,NaN,Warning,True,0-15 Min,True,KA24UZ8488


In [7]:
df['search_type'] = df['search_type'].fillna('Unknown')
df

,stop_date,stop_time,country_name,driver_gender,driver_age,driver_race,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,00:00:00,Canada,M,19,Asian,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,00:01:00,India,M,58,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,00:02:00,USA,M,76,Black,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,00:03:00,Canada,M,76,Black,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,00:04:00,Canada,M,75,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,48,Black,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568
65534,2020-02-15,12:14:00,Canada,F,35,Hispanic,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098
65535,2020-02-15,12:15:00,USA,M,41,Asian,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328
65536,2020-02-15,12:16:00,Canada,F,63,Black,Other,False,Unknown,Warning,True,0-15 Min,True,KA24UZ8488


In [8]:
df.isnull().sum()

stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age            0
driver_race           0
violation             0
search_conducted      0
search_type           0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
dtype: int64

In [9]:
mydb = mysql.connector.connect(
 host="localhost",
 user="root",
 password="",
)
print(mydb)
mycursor = mydb.cursor(buffered=True)

In [10]:
mycursor.execute("CREATE DATABASE IF NOT EXISTS TS")

In [11]:
mycursor.execute("use TS")

sql = """
CREATE TABLE IF NOT EXISTS traffic_stops (
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(10),
    driver_age INT,
    driver_race VARCHAR(50),
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(200),
    stop_outcome VARCHAR(100),
    is_arrested BOOLEAN,
    stop_duration VARCHAR(50),
    drugs_related_stop BOOLEAN,
    vehicle_number VARCHAR(20)
)
"""
    
mycursor.execute(sql)

mydb.commit()

In [12]:
for _, row in df.iterrows():
    insert_sql = """
    INSERT INTO traffic_stops (
        stop_date, stop_time, country_name, driver_gender,
        driver_age, driver_race, violation, search_conducted,
        search_type, stop_outcome, is_arrested, stop_duration,
        drugs_related_stop, vehicle_number
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = (
        row['stop_date'], row['stop_time'], row['country_name'], row['driver_gender'],
        int(row['driver_age']),
        row['driver_race'], row['violation'], bool(row['search_conducted']),
        row['search_type'], row['stop_outcome'], bool(row['is_arrested']),
        row['stop_duration'], bool(row['drugs_related_stop']), row['vehicle_number']
    )
    mycursor.execute(insert_sql, values)

mydb.commit()